<a href="https://colab.research.google.com/github/annasvenbro/etudesnordiques/blob/main/Test_API_SRU_Sudoc_langue_folium.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Test de l'API SRU du Sudoc pour la présence des fonds en langues étrangères dans les RCR du réseau : version utilisant Folium

##Import des paquets nécessaires

In [ ]:
import requests as rq
import xml.etree.ElementTree as et
import pandas as pd
import numpy as np

## Récupérer les données RCR du Sudoc

###Requêter le jeu de données en *open data* d'IdRef pour la liste des RCR via le *webservice* "listrcr" de l'ABES (jeu de données complet *a priori*, avec 2957 entrées).

In [ ]:
liste_rcr=rq.get("https://www.idref.fr/services/listrcr") #On requête l'URL du webservice.
liste_rcr_text=liste_rcr.text #On voit la tête de la réponse.
liste_rcr_text

In [ ]:
lines=liste_rcr_text.split("\n")#Transformation de la réponse en tableau.
header=lines[0].split("\t")
header[0]=header[0].strip("\ufeff") #Pour ne pas avoir de bug dans le dataframe final avec les BOM.
data=[line.split("\t") for line in lines[1:] if line]
df_rcr=pd.DataFrame(data,columns=header) #Transformation en dataframe.
df_rcr

In [ ]:
df_rcr.columns=df_rcr.columns.str.strip("\ufeff") #Nettoyage du dataframe (BOM, signe égal, guillemets et autres caractères parasites).
df_rcr["RCR"]=df_rcr["RCR"].str.replace("=","")
df_rcr["RCR"]=df_rcr["RCR"].str.replace('"','')
df_rcr["PPN"]=df_rcr["PPN"].str.replace("=","")
df_rcr["PPN"]=df_rcr["PPN"].str.replace('"','')
df_rcr= df_rcr.rename(columns={"LONGITUDE\r":"LONGITUDE"})
df_rcr["LONGITUDE"]=df_rcr["LONGITUDE"].str.rstrip("\r")
df_rcr

###Sélection des données pertinentes destinées à alimenter le *dataframe* par RCR à construire pour une langue donnée

Maintenant, il faut ne retenir dans le *dataframe* que 1. le n°RCR de ces bibliothèques, 2. leur PPN (utile pour les infobulles ultérieurement), 3. le nom complet, 4. les coordonnées géographiques.

In [ ]:
df_rcr=df_rcr.filter(regex='^RCR$|^LIBELLE$|^PPN$|^LATITUDE$|^LONGITUDE$')#Sinon on a un bug à cause des BOM.
df_rcr

##Établir un *dataframe* avec tous les résultats par RCR pour une langue donnée

###Création de la fonction de requête en fonction du numéro RCR et de la langue

In [ ]:
langue_fr=input("Quelle est la langue dont vous souhaiteriez obtenir une cartographie des fonds dans le Sudoc ? ")#On pose la question de la langue à requêter.

In [ ]:
#À cette variable, on va en associer une autre correspondant au code ISO 639-2 dont se sert l'API du Sudoc pour ses codes de langue.
langues=rq.get("https://www.loc.gov/standards/iso639-2/ISO-639-2_utf-8.txt") #On va donc créer un dataframe avec les codes de langues, à partir de la liste publiée par la Library of Congress.
langues.encoding="utf-8"
langues_text=langues.text.lstrip("\ufeff") #Encore une fois, pour ne pas avoir de bug dans le dataframe final avec les BOM.
langues_text

In [ ]:
lines=langues_text.split("\n") #Création du dataframe des différentes langues.
data_list=[]
for line in lines:
    if line.strip() != "":
        columns = line.split("|")
        data_list.append({
            "Code":columns[0],
            "Bibliographic":columns[1],
            "Terminology":columns[2],
            "French":columns[4]
        })
df_langues=pd.DataFrame(data_list)
df_langues

In [ ]:
langue=df_langues[df_langues["French"].str.lower().str.contains(langue_fr.lower())]["Code"].values[0]
langue

Attention !!! API du Sudoc distingue 10 langues pour lesquelles il faut utiliser la limitation LAN au lieu de LAI dans la requête API. On va donc devoir faire une disjonction entre les langues qu'il faudra requêter avec le code "LAN" et celle avec le code "LAI".

In [ ]:
LAN=["ger","eng","spa","fre","ita","lat","dut","pol","por","rus"] #On définit la liste des codes de langues centrales devant être requêtées avec le code "LAN" dans l'API du Sudoc.

In [ ]:
def get_langue_sudoc(RCR,langue):
  if langue in LAN:
   req=rq.get(f"https://www.sudoc.abes.fr/cbs/sru/?operation=searchRetrieve&version=1.1&recordSchema=unimarc&query=rbc%3D{RCR}%20and%20lan%3D%22{langue}%22")
  else:req=rq.get(f"https://www.sudoc.abes.fr/cbs/sru/?operation=searchRetrieve&version=1.1&recordSchema=unimarc&query=rbc%3D{RCR}%20and%20lai%3D%22{langue}%22")
  root_sudoc=et.fromstring(req.content)
  for child in root_sudoc.findall("{http://www.loc.gov/zing/srw/}numberOfRecords"):
    return child.text

In [ ]:
get_langue_sudoc(751052115,langue) #On teste la fonction pour la Nordique.

###Création d'un *dataframe* des résultats pour tous les RCR pour une langue donnée (celle donnée par la réponse à la question "Quelle est la langue dont vous souhaiteriez obtenir une cartographie des fonds dans le Sudoc ? ")

In [ ]:
df_rcr["Notices"]=df_rcr.apply(lambda row: get_langue_sudoc(row["RCR"],langue), axis=1)
df_rcr

La mise à jour du *dataframe* prend un certain temps (30 à 40 minutes)...

In [ ]:
df_rcr.dtypes #Ce n'est toujours pas propre pour les notices, qui ne sont pas au format numérique.

In [ ]:
df_rcr["Notices"]=pd.to_numeric(df_rcr["Notices"]) #On veut que cette colonne contienne des données numériques.
df_rcr.dtypes# On vérifie.

In [ ]:
df_rcr=df_rcr[df_rcr["Notices"]!=0] #On supprime les lignes des RCR qui n'ont pas de notices dans la langue concernée.
df_rcr

In [ ]:
df_rcr=df_rcr.sort_values(["Notices"],ascending=False)#On retrie le dataframe pour afficher d'abord les RCR ayant le plus grand nombre de documents.
df_rcr

###Ajout des données pertinentes concernant chaque RCR repéré (adresse du catalogue, mail)

#### Définition de la fonction permettant d'obtenir l'adresse du catalogue et le mail pour chaque RCR (à insérer dans les infobulles plus tard)

In [ ]:
def get_cat_mail(PPN):
  req_cat_mail=rq.get(f"https://www.idref.fr/{PPN}.xml")#Pour un PPN de RCR donné, on obtient sa notice dans IdRef au format XML.
  root_cat_mail=et.fromstring(req_cat_mail.content)
  cat_elt=root_cat_mail.find(".//datafield[@tag='270']//subfield[@code='a']") #On extrait le contenu de la zone avec l'adresse du catalogue du RCR.
  mail_elt=root_cat_mail.find(".//datafield[@tag='220']//subfield[@code='d']") #On extrait le contenu de la zone renseignant l'adresse mail de contact associée au RCR.
  if cat_elt is not None:
    cat=cat_elt.text
  else:
    cat=None
  if mail_elt is not None:
    mail=mail_elt.text
  else:
    mail=None

  if cat is not None and mail is not None: #La fonction retourne l'adresse du catalogue et l'adresse mail associées à chaque PPN de RCR.
    return cat, mail
  elif cat is None:
    return "Adresse catalogue manquante", mail
  else:
    return cat, "Adresse mail manquante"

In [ ]:
get_cat_mail("050960164") #On teste pour la Nordique.

In [ ]:
df_rcr[["Adresse catalogue","Adresse mail"]]=df_rcr["PPN"].apply(lambda x:pd.Series(get_cat_mail(x))) #On rajoute deux colonnes au dataframe précédemment créé, l'une avec l'adresse du catalogue du RCR, l'autre avec son adresse mail de contact.
df_rcr

##Représentations cartographiques et diagrammes

###Création du *geodataframe* et carte des RCR qui ont des notices dans la langue concernée

####Installation et importation des paquets nécessaires à la cartographie

In [ ]:
pip install geopandas

In [ ]:
pip install jenkspy #Installation de la bibliothèque permettant des regroupements naturels entre bibliothèques par nombre de notices grâce à l'agorithme de Jenks.

In [ ]:
import geopandas as gpd
from shapely.geometry import Point
from pyproj import CRS
import folium
from folium.plugins import MarkerCluster #Import du paquet permettant d'agréger géographiquement des groupes de bibliothèques dans la visualisation zoomable permise par Folium.
from jenkspy import jenks_breaks #Import du paquet permettant d'utiliser les regroupements naturels entre bibliothèques.
import matplotlib.pyplot as plt
from datetime import datetime

####Préparation des données du dataframe précédent et création du geodataframe

In [ ]:
df_rcr["LATITUDE"].replace("null",None,inplace=True) #On doit s'occuper des RCR qui n'ont pas de données de géolocalisation.
df_rcr["LONGITUDE"].replace("null",None,inplace=True)

In [ ]:
def create_point(row):
    latitude=float(row["LATITUDE"]) if row["LATITUDE"] is not None else None
    longitude=float(row["LONGITUDE"]) if row["LONGITUDE"] is not None else None
    return Point(longitude,latitude) if latitude and longitude else None

In [ ]:
df_rcr["geometry"]=df_rcr.apply(create_point,axis=1)

In [ ]:
gdf=gpd.GeoDataFrame(df_rcr,geometry="geometry")

In [ ]:
gdf

In [ ]:
print(gdf.crs)#Le CRS n'est pas défini pour le geodataframe !

In [ ]:
gdf.set_crs(epsg=4326,inplace=True) #On définit bien la colonne "geometry" avec le CRS classique "longitude/latitude".

####Export en csv historicisé

In [ ]:
aujourdhui=datetime.now().strftime("%Y%m%d")
nom_fichier=f"{aujourdhui}_cabestan_export_{langue}.csv"
gdf.to_csv(nom_fichier,index=False)

###Carte zoomable pondérée en fonction du nombre de notices avec Folium

Visualisation cartographique avec des données quantitatives concernant le nombre de notices


In [ ]:
carte_pond=folium.Map(location=[46.603354, 1.888334],zoom_start=6)
marker_cluster=MarkerCluster().add_to(carte_pond)
notices_val=gdf["Notices"].values
breaks=jenks_breaks(notices_val,n_classes=3) #Définition de trois classes selon les seuils naturels de l'algorithme de Jenks.

def couleur(notices): #Définition de la fonction définissant les couleurs associées à ces 3 classes (vert : RCR ayant le nombre de notices le plus bas, orange : classe intermédiaire, rouge : RCR ayant le nombre de notices le plus élevé).
  if notices<=breaks[1]:
    return "green"
  elif breaks[1]<notices<=breaks[2]:
    return "orange"
  else:
    return "red"

for idx,row in gdf.iterrows():
    if row.geometry: #Création des infobulles avec : 1. le nom du RCR, 2. son nombre de notices, 3. l'adresse de son catalogue, 4. l'adresse mail de contact.
      popup_content=f"<b>{row['LIBELLE']}</b><br>" \
                    f"Notices:{row['Notices']}<br>" \
                    f"Adresse catalogue: <a href='{row['Adresse catalogue']}' target='_blank'>{row['Adresse catalogue']}</a><br>" \
                    f"Adresse mail: <a href='mailto:{row['Adresse mail']}'>{row['Adresse mail']}</a>"
      folium.Marker(
          location=[row.geometry.y, row.geometry.x],
          popup=folium.Popup(popup_content, max_width=300),
          tooltip=row["LIBELLE"],
          icon=folium.Icon(color=couleur(row["Notices"]))
      ).add_to(marker_cluster) #Création de la carte zoomable.
carte_pond


In [ ]:
carte_pond.save(f"{aujourdhui}_cabestan_carte_ponderee_{langue}.html")

###Diagramme en barres des 25 premières bibliothèques posssédant des fonds dans la langue choisie en termes de nombre de notices

In [ ]:
df_top_25=df_rcr.sort_values("Notices",ascending=False).head(25) #On veut le top 25 du Sudoc en termes de nombres de notices.
df_top_25=df_top_25.iloc[::-1] #On veut une présentation à l'horizontale avec les RCR ayant le plus grand nombre de notices en haut.
date_aujourdhui=datetime.today().strftime("%d-%m-%Y") #On historicise le diagramme en barres.
plt.figure(figsize=(10, 8))
bars=plt.barh(df_top_25["LIBELLE"], df_top_25["Notices"]) #On met le nom des RCR plutôt que leur numéro.
for bar in bars:
    plt.text(bar.get_width(),bar.get_y()+bar.get_height()/2,
             f"{bar.get_width():,.0f}",
             va="center",ha="left")
plt.xlabel("Nombre de notices")
plt.ylabel("Bibliothèques")
plt.title(f"Top 25 des RCR par nombre de notices de documents en {langue_fr} le {date_aujourdhui}")
plt.tight_layout()
plt.show()

In [ ]:
plt.clf()
plt.savefig(f"{aujourdhui}_cabestan_top_25_{langue}.jpg",dpi=300,bbox_inches="tight")